In [ ]:
import pandas as pd
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
import sys

#Mount your Google drive to the VM
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
sys.path.append('/content/gdrive/My Drive/Datasets/')
import os
os.chdir("/content/gdrive/Shared drives/FIT5149/Datasets/")

# Importing libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # for feature extraction


In [ ]:
clean_data = pd.read_csv('preprocessed_label.csv')
clean_data.shape

(49994, 9)

In [ ]:
clean_data.head()

,text,label,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,the new rule is if you are waiting for table w...,4,616,111,-0.208864,1,0,0,1
1,flirted with giving this two star but that a p...,3,1124,206,0.107488,0,0,0,0
2,i wa staying at planet hollywood across the st...,5,593,111,0.364722,0,1,0,0
3,food is good but price are super expensive buc...,2,706,130,0.030485,0,0,0,1
4,worse company to deal with they do horrible wo...,1,630,124,-0.172321,0,0,0,0


In [ ]:
clean_data.label = clean_data.label.astype(int).astype('category')
clean_data.text = clean_data.text.astype(str)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49994 entries, 0 to 49993
Data columns (total 9 columns):
text                  49994 non-null object
label                 49994 non-null category
Text Length           49994 non-null int64
Word Count            49994 non-null int64
Sentiment Polarity    49994 non-null float64
Capital Words         49994 non-null int64
Exclamation           49994 non-null int64
Emoticons             49994 non-null int64
Ques Marks            49994 non-null int64
dtypes: category(1), float64(1), int64(6), object(1)
memory usage: 3.1+ MB


# Feature Extraction

In [ ]:
vectorizer = TfidfVectorizer(max_features=1500,sublinear_tf=True,norm='l2', ngram_range=(1,3), analyzer = "word",  use_idf= True)
X = vectorizer.fit_transform(clean_data['text'])
X.shape

(49994, 1500)

In [ ]:
# convert the vector object to dataframe and append the column names with features.
columns = vectorizer.get_feature_names()
df = pd.DataFrame(X.toarray())
df.columns = columns
print(df.shape)

(49994, 1500)


In [ ]:
df.head()

,able,able to,about,about it,about minute,about the,about this,above,absolutely,across,actually,add,added,after,after the,again,ago,all,all in,all of,all the,almost,along,already,also,although,always,am,amazing,ambiance,amount,amount of,an,an hour,and,and all,and asked,and cheese,and didn,and even,...,won be,wonderful,word,work,worked,working,worse,worst,worth,worth it,worth the,would,would be,would have,would not,wouldn,wow,write,wrong,year,yelp,yes,yet,you,you are,you can,you don,you get,you have,you have to,you ll,you need,you re,you want,you want to,you will,young,your,yourself,yummy
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130426,0.159508,0.00000,0.0,0.0,0.103578,0.106299,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.134739,0.0,0.0,0.0,0.0,0.169144,0.129799,0.112678,0.0,0.0,0.135669,0.16479,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.104579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050825,0.0,0.077522,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.081448,0.079339,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.065681,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.117841,0.118429,0.0,0.0,0.0,0.0,0.0,0.0,0.120732,0.132864,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.12063,0.0,0.0,0.000000,0.090590,0.098093,0.0,0.0,0.0,0.000000,0.0,0.093513,0.0,0.0,0.143967,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.051633,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.093595,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.114928,0.0,0.0,0.0,0.0,0.108454,0.000000,0.162730,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.105278,0.0,0.0,0.0,0.0,0.075074,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.114076,0.0,0.072170,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.197783,0.0,0.0,0.142288,0.0,0.0,0.0,0.0,0.063769,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.143564,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.141494,0.000000,0.0,0.0,0.0,0.0,0.126484,0.0,0.0


In [ ]:
df['Sentiment'] = clean_data['Sentiment Polarity']
df['Capital'] = clean_data['Capital Words']
df['Exclamation'] = clean_data['Exclamation']
df['Question'] = clean_data['Ques Marks']
df['Emoticons'] = clean_data['Emoticons']
df['Text Length'] = clean_data['Text Length']
df['Word Count'] = clean_data['Word Count']

In [ ]:
df.head()

,able,able to,about,about it,about minute,about the,about this,above,absolutely,across,actually,add,added,after,after the,again,ago,all,all in,all of,all the,almost,along,already,also,although,always,am,amazing,ambiance,amount,amount of,an,an hour,and,and all,and asked,and cheese,and didn,and even,...,worst,worth,worth it,worth the,would,would be,would have,would not,wouldn,wow,write,wrong,year,yelp,yes,yet,you,you are,you can,you don,you get,you have,you have to,you ll,you need,you re,you want,you want to,you will,young,your,yourself,yummy,Sentiment,Capital,Exclamation,Question,Emoticons,Text Length,Word Count
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130426,0.159508,0.00000,0.0,0.0,0.103578,0.106299,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.134739,0.0,0.0,0.0,0.0,0.169144,0.129799,0.112678,0.0,0.0,0.135669,0.16479,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-0.208864,1,0,1,0,616,111
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.104579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050825,0.0,0.077522,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.081448,0.079339,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.065681,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.107488,0,0,0,0,1124,206
2,0.117841,0.118429,0.0,0.0,0.0,0.0,0.0,0.0,0.120732,0.132864,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.12063,0.0,0.0,0.000000,0.090590,0.098093,0.0,0.0,0.0,0.000000,0.0,0.093513,0.0,0.0,0.143967,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.051633,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.364722,0,1,0,0,593,111
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.093595,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.114928,0.0,0.0,0.0,0.0,0.108454,0.000000,0.162730,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.105278,0.0,0.0,0.0,0.0,0.075074,0.0,0.0,0.030485,0,0,1,0,706,130
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.114076,0.0,0.072170,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.063769,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.143564,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.141494,0.000000,0.0,0.0,0.0,0.0,0.126484,0.0,0.0,-0.172321,0,0,0,0,630,124


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = pd.DataFrame(df, columns=df.columns)
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
# x[['Capital', 'Exclamation', 'Question', 'Emoticons', 'Text Length', 'Word Count']] = scaler.fit_transform(x[['Capital', 'Exclamation', 'Question', 'Emoticons', 'Text Length', 'Word Count']])
y = clean_data["label"]

## Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split
import random
# split the dataset to train and test with test size 20%. 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 5) 

# Models

## Logistic Regression

In [ ]:
from sklearn import linear_model

# Multinomial Logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train,y_train)

In [ ]:
from sklearn import metrics

#test the model to predict the test set.
pred = mul_lr.predict(X_test)

# determine the accuracy
metrics.accuracy_score(y_test,pred)

0.5894589458945895

In [ ]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

C_param_range = [0.1, 0.2, 0.3, 0.6, 1, 10, 100]

for c in C_param_range:
    # Apply logistic regression model to training data
    lr = linear_model.LogisticRegression(multi_class='multinomial',
                                         solver='newton-cg', class_weight="balanced", 
                                         fit_intercept=False, tol = 0.00001,
                                         penalty = 'l2', C = c , random_state = 5)
    lr.fit(X_train,y_train)

    # Predict using model
    pred = lr.predict(X_test)

    # Saving accuracy score in table
    accuracy = metrics.accuracy_score(y_test,pred)
    print("\nModel Report")
    print("C = ", c, " Accuracy : ",  accuracy)
    print()


Model Report
C =  0.1  Accuracy :  0.5889588958895889


Model Report
C =  0.2  Accuracy :  0.5916591659165916


Model Report
C =  0.3  Accuracy :  0.590959095909591


Model Report
C =  0.6  Accuracy :  0.5862586258625863


Model Report
C =  1  Accuracy :  0.5835583558355836


Model Report
C =  10  Accuracy :  0.5812581258125813


Model Report
C =  100  Accuracy :  0.581058105810581



#NB

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train, y_train)

#test the model to predict the test set.
pred = clf.predict(X_test)

# determine the accuracy
print(metrics.accuracy_score(y_test,pred))

0.5427542754275427


# SVM

In [ ]:
from sklearn.svm import LinearSVC
LSVC = LinearSVC()
LSVC.fit(X_train,y_train)
pred = LSVC.predict(X_test)
print("Accuracy of SVM",metrics.accuracy_score(y_test,pred))

Accuracy of SVM 0.5721572157215722


# Ensembling of SVM, NB and Logistic Regression

In [ ]:
from sklearn.ensemble import VotingClassifier
seed = 7
# kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
# model1 = LogisticRegression()
estimators.append(('logistic', mul_lr))
# model2 = DecisionTreeClassifier()
estimators.append(('MultinomialNB', clf))
# model3 = SVC()
estimators.append(('svm', LSVC))
# create the ensemble model
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
pred = ensemble.predict(X_test)
# results = model_selection.cross_val_score(ensemble, x, y, cv=kfold)
# print(results.mean())

In [ ]:
print("Ensemble",  metrics.accuracy_score(y_test, pred))

Ensemble 0.5828582858285829


## test data

In [ ]:
data = pd.read_csv('preprocessed_test.csv')
data.shape

(50000, 9)

In [ ]:
data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,test_1,trying to have nice quiet dinner the announcer...,109,20,0.233333,0,0,0,0
1,test_2,been getting food to go from here for over yr ...,659,124,0.158929,0,0,0,0
2,test_3,ugh ve had to eat here couple of time because ...,1335,248,0.047309,7,3,0,0
3,test_4,the people here are so nice ordered on eat and...,174,32,0.337500,0,3,1,0
4,test_5,heard alot of good thing about this place and ...,529,95,0.304625,0,2,0,0


In [ ]:
data.text = data.text.astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
test_id               50000 non-null object
text                  50000 non-null object
Text Length           50000 non-null int64
Word Count            50000 non-null int64
Sentiment Polarity    50000 non-null float64
Capital Words         50000 non-null int64
Exclamation           50000 non-null int64
Ques Marks            50000 non-null int64
Emoticons             50000 non-null int64
dtypes: float64(1), int64(6), object(2)
memory usage: 3.4+ MB


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_2 = vectorizer.transform(data['text'])
X_2.shape

(50000, 1500)

In [ ]:
# convert the vector object to dataframe and append the column names with features.
columns = vectorizer.get_feature_names()
df_2 = pd.DataFrame(X_2.toarray())
df_2.columns = columns
print(df_2.shape)

(50000, 1500)


In [ ]:
df_2['Sentiment'] = data['Sentiment Polarity']
df_2['Capital'] = data['Capital Words']
df_2['Exclamation'] = data['Exclamation']
df_2['Question'] = data['Ques Marks']
df_2['Emoticons'] = data['Emoticons']
df_2['Text Length'] = data['Text Length']
df_2['Word Count'] = data['Word Count']

In [ ]:
# x_2 = pd.DataFrame(df_2, columns=df_2.columns)
x_2 = pd.DataFrame(scaler.transform(df_2), columns=df_2.columns)

In [ ]:
# x_2[['Capital', 'Exclamation', 'Question', 'Emoticons', 'Text Length', 'Word Count']] = scaler.fit_transform(x_2[['Capital', 'Exclamation', 'Question', 'Emoticons', 'Text Length', 'Word Count']])

In [ ]:
pred = mul_lr.predict(x_2)

In [ ]:
data['label']=list(pred)

In [ ]:
x_2.head()

,able,absolutely,across,actual,actually,add,added,addition,additional,afternoon,again,ago,ahead,air,airport,almost,alone,along,already,alright,also,also not,also very,although,always,amazing,ambiance,american,amount,annoying,another,answer,anymore,anyone,anything,anyway,anywhere,apartment,apparently,appetizer,...,worker,working,world,worse,worst,worth,would,would come,would definitely,would give,would go,would like,would never,would not,would not recommend,would recommend,would say,wow,wrap,write,wrong,yeah,year,year ago,year old,yelp,yes,yesterday,yet,young,yum,yummy,zero,Sentiment,Capital,Exclamation,Question,Emoticons,Text Length,Word Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.616667,0.00,0.000000,0.0,0.000000,0.021155,0.018609
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.187978,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.579464,0.00,0.000000,0.0,0.000000,0.129893,0.120470
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.275746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162354,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.249718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.208143,0.0,0.0,0.0,0.0,0.0,0.0,0.210919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.523654,0.07,0.035294,0.0,0.000000,0.263543,0.241920
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.668750,0.00,0.035294,0.0,0.166667,0.034006,0.030362
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227246,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652312,0.00,0.023529,0.0,0.000000,0.104191,0.092067


In [ ]:
data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Ques Marks,Emoticons,label
0,test_1,trying nice quiet dinner announcer award givea...,109,20,0.233333,0,0,0,0,3
1,test_2,getting food go yr wife usually tend get same ...,659,124,0.158929,0,0,0,0,4
2,test_3,ugh ve eat couple time work event course make ...,1335,248,0.047309,7,3,0,0,1
3,test_4,people nice ordered eat promptly called double...,174,32,0.337500,0,3,0,1,5
4,test_5,heard alot good thing place decided grab break...,529,95,0.304625,0,2,0,0,4


In [ ]:
kaggle=data[['test_id','label']]

In [ ]:
kaggle.to_csv('kaggle1.csv', encoding='utf-8', index=False)

In [ ]:
kaggle.head()

,test_id,label
0,test_1,3
1,test_2,4
2,test_3,1
3,test_4,5
4,test_5,4


In [ ]:
# data_part1 = unlabeled_data[:149999]
# data_part1.shape

In [ ]:
# data = some_features(data)

In [ ]:
data

In [ ]:
columns = vectorizer.get_feature_names()
df = pd.DataFrame(X.toarray())
df.columns = columns
print(df.shape)

(49994, 1500)


In [ ]:
data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Ques Marks,Emoticons
0,test_1,trying nice quiet dinner announcer award givea...,109,20,0.233333,0,0,0,0
1,test_2,getting food go yr wife usually tend get same ...,659,124,0.158929,0,0,0,0
2,test_3,ugh ve eat couple time work event course make ...,1335,248,0.047309,7,3,0,0
3,test_4,people nice ordered eat promptly called double...,174,32,0.337500,0,3,0,1
4,test_5,heard alot good thing place decided grab break...,529,95,0.304625,0,2,0,0


In [ ]:
# vectorizer = TfidfVectorizer(max_features=1500,sublinear_tf=True,norm='l2',ngram_range=(1,3),analyzer = "word")
# X = vectorizer.fit_transform(data['text'])
# X.shape

In [ ]:
data.text = data.text.astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
test_id    50000 non-null object
text       50000 non-null object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
X = vectorizer.transform(data['text'])


In [ ]:
# convert the vector object to dataframe and append the column names with features.
columns = vectorizer.get_feature_names()
df = pd.DataFrame(X.toarray())
df.columns = columns
print(df.shape)

(50000, 10000)


In [ ]:
df['Sentiment'] = data['Sentiment Polarity']
df['Capital'] = data['Capital Words']
df['Exclamation'] = data['Exclamation']
df['Question'] = data['Ques Marks']
df['Emoticons'] = data['Emoticons']

In [ ]:
df.head()

,ability,able,able find,able get,absolute,absolute worst,absolutely,absolutely amazing,absolutely delicious,absolutely horrible,...,zero star,zero star would,zone,zoo,zucchini,Sentiment,Capital,Exclamation,Question,Emoticons
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.233333,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.158929,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.047309,7,3,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.337500,0,3,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.304625,0,2,0,0


In [ ]:
pred1 = mul_lr.predict(df)

In [ ]:
pred1

array([3, 4, 1, ..., 2, 5, 3], dtype=int64)

In [ ]:
data['label']=list(pred1)
data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Ques Marks,Emoticons,label
0,test_1,trying nice quiet dinner announcer award givea...,109,20,0.233333,0,0,0,0,3
1,test_2,getting food go yr wife usually tend get same ...,659,124,0.158929,0,0,0,0,4
2,test_3,ugh ve eat couple time work event course make ...,1335,248,0.047309,7,3,0,0,1
3,test_4,people nice ordered eat promptly called double...,174,32,0.337500,0,3,0,1,5
4,test_5,heard alot good thing place decided grab break...,529,95,0.304625,0,2,0,0,4


In [ ]:
data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Ques Marks,Emoticons,label
0,test_1,trying nice quiet dinner announcer award givea...,109,20,0.233333,0,0,0,0,3
1,test_2,getting food go yr wife usually tend get same ...,659,124,0.158929,0,0,0,0,4
2,test_3,ugh ve eat couple time work event course make ...,1335,248,0.047309,7,3,0,0,1
3,test_4,people nice ordered eat promptly called double...,174,32,0.337500,0,3,0,1,5
4,test_5,heard alot good thing place decided grab break...,529,95,0.304625,0,2,0,0,4


In [ ]:
data.shape

(50000, 10)

In [ ]:
kaggle=data[['test_id','label']]

In [ ]:
kaggle.head()

,test_id,label
0,test_1,3
1,test_2,4
2,test_3,1
3,test_4,5
4,test_5,4


In [ ]:
kaggle.to_csv('kaggle\kaggle1.csv', encoding='utf-8', index=False)

# Pipeline

In [ ]:
data1 = pd.read_csv('preprocessed_label.csv')
test_data = pd.read_csv('preprocessed_test.csv')
data1.text = data1.text.astype(str)
test_data.text = test_data.text.astype(str)

In [ ]:
unlabeled_data = pd.read_csv('preprocessed_unlabel.csv')
unlabeled_data.text = unlabeled_data.text.astype(str)
unlabeled_data.shape

(598439, 8)

In [ ]:
labeled_data_1 = pd.read_csv('labeled_data.csv')
labeled_data = labeled_data_1.copy()
labeled_data.head()

In [ ]:
import random
from sklearn.model_selection import train_test_split
random.seed(555)
X_train, X_test, y_train, y_test = train_test_split(labeled_data.text, labeled_data.label, test_size = 0.3) 

In [ ]:
X_test

26246    cooling down ll heading back maizie summertime...
22571    first time location service employee great fri...
17689    delicious place very friendly folk many option...
6891     great sushi however preferred old location foo...
48886    decided visit despite warned hotel concierge g...
                               ...                        
10696    few staple toronto burrito boyz one used joine...
42163    go breakfast whenever near downtown tend gravi...
17860    third la palmas market taqueria pittsburgh rev...
47805    ve twice not eaten rather come micro brewery l...
33471    saturday special stew chicken rice pea maybe s...
Name: text, Length: 14999, dtype: object

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import linear_model
lg = Pipeline([('vect', CountVectorizer(ngram_range = (1,3))),
              ('tfidf', TfidfTransformer(use_idf = True)),
              ('clf', linear_model.LogisticRegression(multi_class='multinomial', solver = 'newton-cg', C = 0.2, tol = 0.00001))])

In [ ]:
lg.fit(X_train, y_train)
pred = lg.predict(X_test)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test,pred)

0.5753716914460965

In [ ]:
for c in [0.01, 0.1, 0.2, 0.5, 1, 10]:
  lg = Pipeline([('vect', CountVectorizer(ngram_range = (1,3))),
              ('tfidf', TfidfTransformer(use_idf = True)),
              ('clf', linear_model.LogisticRegression(multi_class='multinomial', solver = 'newton-cg', C = c, tol = 0.00001))]) 
  lg.fit(X_train, y_train)
  pred = lg.predict(X_test)
  acc = metrics.accuracy_score(y_test,pred)
  print("\nModel Report")
  print("C = ", c, " Accuracy : ",  acc)
  print()


Model Report
C =  0.01  Accuracy :  0.4931662110807387


Model Report
C =  0.1  Accuracy :  0.5539035935729049


Model Report
C =  0.2  Accuracy :  0.5753716914460965


Model Report
C =  0.6  Accuracy :  0.5993066204413627


Model Report
C =  1  Accuracy :  0.6057737182478832


Model Report
C =  10  Accuracy :  0.6097073138209214



In [ ]:
lg = Pipeline([('vect', CountVectorizer(ngram_range = (1,3))),
              ('tfidf', TfidfTransformer(use_idf = True)),
              ('clf', linear_model.LogisticRegression(multi_class='multinomial', solver = 'newton-cg', C = 100, tol = 0.00001))]) 
lg.fit(X_train, y_train)
pred = lg.predict(X_test)
acc = metrics.accuracy_score(y_test,pred)
print("\nModel Report")
print("C = ", 100, " Accuracy : ",  acc)
print()


Model Report
C =  100  Accuracy :  0.6075738382558837



In [ ]:
unlabeled_10000 = unlabeled_data[1:10000]

In [ ]:
unlabeled_10000['label'] = lg.predict(unlabeled_10000.text)

In [ ]:
frames = [data1, unlabeled_10000]
new_data = pd.concat(frames)

In [ ]:
frames1 = [X_train, unlabeled_10000.text]
new_train = pd.concat(frames1)
frames2 = [y_train, unlabeled_10000.label]
new_train_y = pd.concat(frames2)

In [ ]:
lg.fit(new_train, new_train_y)
pred_new = lg.predict(X_test)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test,pred_new)

0.5981065404360291

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import linear_model
lg = Pipeline([('vect', CountVectorizer(ngram_range = (1,3))),
              ('tfidf', TfidfTransformer(use_idf = True)),
              ('clf', linear_model.LogisticRegression(multi_class='multinomial', solver = 'newton-cg', C = 10))])

In [ ]:
pred1 = lg.predict(test_data.text)

In [ ]:
data1.shape

(49994, 9)

In [ ]:
test_data['label']=list(pred1)
test_data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Ques Marks,Emoticons,label
0,test_1,trying nice quiet dinner announcer award givea...,109,20,0.233333,0,0,0,0,3
1,test_2,getting food go yr wife usually tend get same ...,659,124,0.158929,0,0,0,0,4
2,test_3,ugh ve eat couple time work event course make ...,1335,248,0.047309,7,3,0,0,2
3,test_4,people nice ordered eat promptly called double...,174,32,0.337500,0,3,0,1,5
4,test_5,heard alot good thing place decided grab break...,529,95,0.304625,0,2,0,0,4


In [ ]:
kaggle=test_data[['test_id','label']]

In [ ]:
kaggle.to_csv('kaggle\kaggle1.csv', encoding='utf-8', index=False)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
ovr = Pipeline([('vect', CountVectorizer(ngram_range = (1,3))),
              ('tfidf', TfidfTransformer(use_idf = True)),
              ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced')))])

In [ ]:
ovr.fit(X_train, y_train)
pred = ovr.predict(X_test)

from sklearn import metrics
metrics.accuracy_score(y_test,pred)

0.6010880870469637

In [ ]:
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2), (2,2), (1,3), (2,3), (3,3)],
              'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(ovr, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

ValueError: ignored